<a href="https://colab.research.google.com/github/Vusumuzi2Mabasa/BioHack2022/blob/main/Protein%20engineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Install conda


In [ ]:
! wget https://repo.anaconda.com/miniconda/Miniconda3-py37_4.8.2-Linux-x86_64.sh
! chmod +x Miniconda3-py37_4.8.2-Linux-x86_64.sh
! bash ./Miniconda3-py37_4.8.2-Linux-x86_64.sh -b -f -p /usr/local
import sys
sys.path.append('/usr/local/lib/python3.7/site-packages/')

#Download and Install Pfeature

In [ ]:
! wget https://github.com/raghavagps/Pfeature/raw/master/PyLib/Pfeature.zip

In [ ]:
! unzip Pfeature.zip

In [ ]:
% cd Pfeature

In [ ]:
! python setup.py install

#Install CD-HIT

In [ ]:
! conda install -c bioconda cd-hit -y

# Load peptide dataset

In [ ]:
! wget https://raw.githubusercontent.com/dataprofessor/AMP/main/train_po.fasta

In [ ]:
! cat train_ne.fasta

In [ ]:
! cd-hit -i train_po.fasta -o train_po_cdhit.txt -c 0.99

In [ ]:
! cd-hit -i train_ne.fasta -o train_ne_cdhit.txt -c 0.99

In [ ]:
! ls -l

In [ ]:
! grep ">" train_po_cdhit.txt | wc -l

In [ ]:
! grep ">" train_po.fasta | wc -l

In [ ]:
! grep ">" train_ne.fasta | wc -l

In [ ]:
! grep ">" train_ne_cdhit.txt | wc -l

Calculate features using the Pfeature library
Feature classes provided by Pfeature is summarized in the tables below.

Composition Based Features

Feature claass	Description	Function
AAC	Amino acid composition	aac_wp
DPC	Dipeptide composition	dpc_wp
TPC	Tripeptide composition	tpc_wp
ABC	Atom and bond composition	atc_wp, btc_wp
PCP	Physico-chemical properties	pcp_wp
AAI	Amino acid index composition	aai_wp
RRI	Repetitive Residue Information	rri_wp
DDR	Distance distribution of residues	ddr_wp
PRI	Physico-chemical properties repeat composition	pri_wp
SEP	Shannon entropy	sep_wp
SER	Shannon entropy of residue level	ser_wp
SPC	Shannon entropy of physicochemical property	spc_wp
ACR	Autocorrelation	acr_wp
CTC	Conjoint Triad Calculation	ctc_wp
CTD	Composition enhanced transition distribution	ctd_wp

Define functions for calculating the different features

In [ ]:
import pandas as pd

In [ ]:
# Amino acid composition (AAC)

from Pfeature.pfeature import aac_wp

def aac(input):
  a = input.rstrip('txt')
  output = a + 'aac.csv'
  df_out = aac_wp(input, output)
  df_in = pd.read_csv(output)
  return df_in

aac('train_po_cdhit.txt')

In [ ]:
# Dipeptide composition (DPC)

from Pfeature.pfeature import dpc_wp

def dpc(input):
  a = input.rstrip('txt')
  output = a + 'dpc.csv'
  df_out = dpc_wp(input, output, 1)
  df_in = pd.read_csv(output)
  return df_in

feature = dpc('train_po_cdhit.txt')
feature

#Calculate feature for both positive and negative classes + combines the two classes + merge with class labels

In [ ]:
pos = 'train_po_cdhit.txt'
neg = 'train_ne_cdhit.txt'

def feature_calc(po, ne, feature_name):
  # Calculate feature
  po_feature = feature_name(po)
  ne_feature = feature_name(ne)
  # Create class labels
  po_class = pd.Series(['positive' for i in range(len(po_feature))])
  ne_class = pd.Series(['negative' for i in range(len(ne_feature))])
  # Combine po and ne
  po_ne_class = pd.concat([po_class, ne_class], axis=0)
  po_ne_class.name = 'class'
  po_ne_feature = pd.concat([po_feature, ne_feature], axis=0)
  # Combine feature and class
  df = pd.concat([po_ne_feature, po_ne_class], axis=1)
  return df

feature = feature_calc(pos, neg, aac) # AAC
#feature = feature_calc(pos, neg, dpc) # DPC
feature

#Data pre-processing

In [ ]:
feature

In [ ]:
# Assigns the features to X and class label to Y
X = feature.drop('class', axis=1)
y = feature['class'].copy()

In [ ]:
# Encoding the Y class label
y = y.map({"positive": 1, "negative": 0}) 

In [ ]:
X.shape

In [ ]:
# Feature selection (Variance threshold)
from sklearn.feature_selection import VarianceThreshold

fs = VarianceThreshold(threshold=0.1)
fs.fit_transform(X)
#X2.shape
X2 = X.loc[:, fs.get_support()]
X2

In [ ]:
# Data split
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X2, y, test_size=0.2, random_state =42, stratify=y)

#Quickly compare >30 ML algorithms

In [ ]:
! pip install lazypredict

In [ ]:
# Import libraries
import lazypredict
from lazypredict.Supervised import LazyClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import matthews_corrcoef

# Load dataset
X = feature.drop('class', axis=1)
y = feature['class'].copy()

# Data split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state =42, stratify=y)

# Defines and builds the lazyclassifier
clf = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=matthews_corrcoef)
models_train,predictions_train = clf.fit(X_train, X_train, y_train, y_train)
#models_test,predictions_test = clf.fit(X_train, X_test, y_train, y_test)

In [ ]:
# Prints the model performance (Training set)
models_train

In [ ]:
# Prints the model performance (Test set)
models_test

In [ ]:
y_test

In [ ]:
# Plot of Accuracy
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(5, 10))
sns.set_theme(style="whitegrid")
ax = sns.barplot(y=models_train.index, x="Accuracy", data=models_train)
ax.set(xlim=(0, 1))

In [ ]:
# Plot of MCC
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(5, 10))
sns.set_theme(style="whitegrid")
ax = sns.barplot(y=models_train.index, x="matthews_corrcoef", data=models_train)
ax.set(xlim=(0, 1))

#Random Forest

In [ ]:
# Build random forest model

from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=500)

rf.fit(X_train, y_train)

#Apply the model to make predictions

In [ ]:
y_train_pred = rf.predict(X_train)
y_test_pred = rf.predict(X_test)

In [ ]:
feature['class']

In [ ]:
# Simplest and quickest way to obtain the model performance (Accuracy)
rf.score(X_test,y_test)

In [ ]:
# Accuracy
from sklearn.metrics import accuracy_score

accuracy_score(y_test, y_test_pred)

In [ ]:
# Matthew Correlation Coefficient
from sklearn.metrics import matthews_corrcoef

matthews_corrcoef(y_test, y_test_pred)

In [ ]:
# Confusion matrix
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, y_test_pred)

In [ ]:
# Classification report
from sklearn.metrics import classification_report

model_report = classification_report(y_train, y_train_pred, target_names=['positive','negative'])

f = open('model_report.txt','w')
f.writelines(model_report) 
f.close()

In [ ]:
# ROC curve
import matplotlib.pyplot as plt
from sklearn.metrics import plot_roc_curve

plot_roc_curve(rf, X_test, y_test)  
plt.show()

In [ ]:
plot_roc_curve(rf, X_train, y_train)  
plt.show()

#Feature importance

In [ ]:
# Display Dataframe of the dataset after feature selection (variance threshold)
X2

In [ ]:
# Retrieve feature importance from the RF model
importance = pd.Series(rf.feature_importances_, name = 'Gini')

# Retrieve feature names
feature_names = pd.Series(X2.columns, name = 'Feature')

In [ ]:
# Combine feature names and Gini values into a Dataframe
df = pd.concat([feature_names, importance], axis=1, names=['Feature', 'Gini'])
df

In [ ]:
# Plot of feature importance
import matplotlib.pyplot as plt
import seaborn as sns

df_sorted = df.sort_values('Gini', ascending=False)[:20] # Sort by Gini in descending order; Showing only the top 20 results

plt.figure(figsize=(5, 10))
sns.set_theme(style="whitegrid")
ax = sns.barplot(x = 'Gini', y = 'Feature', data = df_sorted)
plt.xlabel("Feature Importance")
